In [ ]:
include("init.jl")

In [ ]:
import Neo4j

In [ ]:
c = Neo4j.Connection("localhost"; user="neo4j", password=ENV["NEO4JPW"]);

In [ ]:
#tx = Neo4j.transaction(c)
#tx("""
#    MATCH (n:Repository)-->()-->(m)
#    RETURN toInt(n.id),toInt(m.id)
#    LIMIT 10
#""")
#results = Neo4j.commit(tx)

In [ ]:
# Don't run this
# I feel like there must be an "end statement" proword, to save having this silly map stuff.
import_query = [
    """
        match (n) detach delete n
    """,
    """
        LOAD CSV WITH HEADERS FROM "file:///home/olie/Dropbox_work/big_data/libraries-io/playground/julia_projs_deps.csv" as line
        CREATE (n:Dependency)
        SET n = line
    """,
    """
        LOAD CSV WITH HEADERS FROM "file:///home/olie/Dropbox_work/big_data/libraries-io/playground/julia_repos.csv" as line
        CREATE (n:Repository)
        SET n = line
    """,
    """
        LOAD CSV WITH HEADERS FROM "file:///home/olie/Dropbox_work/big_data/libraries-io/playground/julia_projs.csv" as line
        CREATE (n:Project)
        SET n = line
    """,
    """
        match (n:Project),(m:Dependency)
        where m.`Repository ID` = n.`Repository ID`
        create (n)-[:DEPENDENCY]->(m)
    """,
    """
        match (n:Project),(m:Dependency)
        where m.`Dependency Project ID` = n.`ID`
        create (m)-[:DEPENDS_ON]->(n)
    """,
    """
        match (n:Repository),(m:Dependency)
        where m.`Repository ID` = n.`ID`
        create (n)-[:DEPENDENCY]->(m)
    """,
    """
        match (n:Repository),(m:Project)
        where m.`Repository ID` = n.`ID`
        create (m)-[:REPO]->(n) 
    """,
]

tx = Neo4j.transaction(c)
map(tx,import_query)
results = Neo4j.commit(tx)

In [ ]:
results = Neo4j.cypherQuery(c, """
    MATCH (n)-->()-->(m)
    WHERE (n:Repository OR n:Project) AND (m:Repository OR m:Project)
    RETURN toInt(n.ID),toInt(m.ID)
""")

In [ ]:
import LightGraphs, DataFrames

In [ ]:
nodelist = Set([results[1:size(results,1),1] results[1:size(results,1),2]])
nodemap = Dict(v => k for (k,v) in enumerate(nodelist))

In [ ]:
g = LightGraphs.DiGraph(nodelist |> length)
for r in DataFrames.eachrow(results)
    LightGraphs.add_edge!(g,(nodemap[r[1]],nodemap[r[2]]))
end
g

# We can probably speed this up a lot / use a StaticGraph
# but is fine for now